In [ ]:
import os
import json
import numpy as np
import pandas as pd

In [ ]:
Home = os.getcwd()
DataFolder = os.path.join(Home,'Data')
df = pd.read_csv(os.path.join(DataFolder,'MyDataSet.csv'))
df_cols = list(df.columns)
df = df[df.filter(regex='^(?!Unnamed)').columns]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from niapy.algorithms import basic

In [ ]:
def Define_Model():
    model = RandomForestClassifier()
    param_grid = {}
    return model, param_grid

In [ ]:
df_cols = list(df.columns)
y = df[df_cols[-1]]
X = df[df_cols[:-1]]
start_val = 0
model, param_grid = Define_Model()

In [ ]:
def SKLEARN_CUCKOOSEARCH(Kmodel, Kparam_grid):
    lst = []
    for Kpopulation_size in range(10,101,10): #10
        for Kpa in np.arange(0.01,1.01,.01): #100
            
            Algo = basic.cs.CuckooSearch()
            Algo.set_parameters(
                population_size = Kpopulation_size,
                pa = Kpa
            )
            NIS = NatureInspiredSearchCV(
                estimator = Kmodel,
                param_grid = Kparam_grid,
                algorithm=Algo
            )
            lst.append(NIS)
    return lst

In [ ]:
def Save_Often(CS_Dict,Name):
    ResultFolder = os.path.join(Home, 'Result')
    Name = os.path.join(ResultFolder, Name)
    with open(Name,'w') as kanaishk:
        json.dump(CS_Dict,kanaishk)

In [ ]:
NIS_lst = SKLEARN_CUCKOOSEARCH(model, param_grid)
print(len(NIS_lst))
Best_Score = 0
Best_Para = None 
CS_Dict = dict()

In [ ]:
for i in range(len(NIS_lst)):
    if(i%100==0 and i != start_val):
        Name = f'Kanaishk Cuckoo Search Dict-{i/100}.json'
        Save_Often(CS_Dict,Name)
        CS_Dict = dict()
    Curr_Candidate = NIS_lst[i]
    Curr_Candidate.fit(X,y)
    Curr_Para = NIS_lst[i].algorithm.get_parameters()
    Curr_Score = Curr_Candidate.best_score_
    CS_Dict[str(Curr_Para)] = Curr_Score
    if Curr_Score > Best_Score:
        Best_Para = Curr_Para

In [ ]:
#Incase some of the model do not work
'''CS_Dict = dict()
for i in range(800,900):
    Curr_Candidate = NIS_lst[i]
    Curr_Candidate.fit(X,y)
    Curr_Para = NIS_lst[i].algorithm.get_parameters()
    Curr_Score = Curr_Candidate.best_score_
    CS_Dict[str(Curr_Para)] = Curr_Score
    if Curr_Score > Best_Score:
        Best_Para = Curr_Para
Name = 'Kanaishk Cuckoo Search Dict-9.0.json'
Save_Often(CS_Dict,Name)'''

In [ ]:
print(Best_Score)
print(Best_Para)

In [ ]:
#Orignal Paper Cuckoo Search: https://ieeexplore.ieee.org/abstract/document/5393690
#Sklearn Dir: https://github.com/timzatko/Sklearn-Nature-Inspired-Algorithms
#NIAPY ALgorithm Class: https://github.com/NiaOrg/NiaPy/blob/master/niapy/algorithms/algorithm.py
#NIAPY Cuckoo Search Algorithm Class: https://github.com/NiaOrg/NiaPy/blob/master/niapy/algorithms/basic/cs.py
#NIAPY How to test Algorithm: https://github.com/NiaOrg/NiaPy/blob/master/niapy/tests/test_algorithm.py
#NIAPY How to test CS: https://github.com/NiaOrg/NiaPy/blob/master/niapy/tests/test_cs.py
#NIAPY Task (Answer to Sphere Function): https://github.com/NiaOrg/NiaPy/blob/master/niapy/task.py